# Создание таблиц из json-файлов размеченных отзывов



In [1]:
import json
import pandas as pd

## Импорт таблицы с упоминаниями лекарственных препаратов и соответствующих им стандартными названиями. Проверка корректности импорта

In [2]:
drugname_df = pd.read_excel('Drugname_2022.xlsx')
drugname_df

,mention,standart,ATC,description,comments,med_NNreview,med_1kADR
0,2 в 1 garnier чистая кожа актив,garnier 2 в 1 чистая кожа актив,NaN,NaN,Косметическое средство,NaN,563469
1,5 нок,5 нок,J01XX07,Nitroxoline,NaN,1383732,NaN
2,5- нок,5 нок,J01XX07,Nitroxoline,NaN,1383732,NaN
3,5нок,5 нок,J01XX07,Nitroxoline,NaN,3145688,NaN
4,"911 ""бодяга",911 бадяга,D11AX,Прочие препараты для лечения заболеваний кожи,NaN,NaN,2702981
...,...,...,...,...,...,...,...
2604,vicks active sinex,vicks active sinex,R01AA05,Oxymetazoline,NaN,NaN,2249637
2605,vitrum,vitrum,A11AA04,Поливитамины с микроэлементами,NaN,NaN,1373327/671606
2606,vitrum kids,vitrum,A11AA04,Поливитамины с микроэлементами,NaN,NaN,4283315
2607,zerosmoke,zerosmoke,N07BA03,Varenicline,NaN,NaN,1175393/1256045/1929179


## Определение вспомогательных функций

In [3]:
def standartization(drug_list):
    standart_drug_list = [0 for i in range(len(drug_list))]
    mention, standart = list(drugname_df.mention), list(drugname_df.standart)
    for i in range(len(drug_list)):
      if drug_list[i] != 0:
          if drug_list[i].lower() in mention:
            indices = [j for j in range(0, len(mention)) if mention[j]==drug_list[i].lower()]
            for ind in indices:
              standart_drug_list[i] = standart[ind]
    return standart_drug_list


def remove_duplicates(list_with_duplicates):
    list2 = []
    for item in list_with_duplicates:
      if item not in list2:
        list2.append(item)
    return list2


def top3_drugs(ordered_list_of_drugs):
    if len(ordered_list_of_drugs) > 3:
        return ordered_list_of_drugs[:3]
    else:
        return ordered_list_of_drugs


def count_frequency(list_for_count):
    dict_with_frequency = {}
    for elem in list_for_count:
        if elem not in dict_with_frequency:
            dict_with_frequency[elem] = 0
        dict_with_frequency[elem] += 1
    return dict_with_frequency

## Парсинг json-файлов:
- cоздание для каждого упоминания лекарственного препарата количества BNE-Pos, ADE-Neg, Worse, NegatedADE, ADR и списка ADR
- создание для каждого симптома списка лекарственных препаратов

In [4]:
with open("results.json", "r") as f:
    data = json.load(f)

names = []
bne_counts = []
ade_neg_counts = []
worse_counts = []
negated_ade_counts = []
adr_counts = []
adr_list = []
diseases = {}

for i in range(len(data)):
    text_id = data[i]['text_id']
    name = 'undefined'
    #for entity in data[i]['entities'].values():
    for entity in data[i]['data']['entities'].values():
        bne_count = 0
        ade_neg_count = 0
        worse_count = 0
        negated_ade_count = 0
        adr_count = 0
        adr_text = 'Нет ADR'
        if 'MedType' in entity:
            if entity["MedType"] == "Drugname":
                name = entity["text"]
                for i in range(len(names)):
                    if names[i] == 'undefined':
                        names[i] = name
        if 'DisType' in entity:
            if entity['DisType'] == "BNE-Pos":
                bne_count += 1

            if entity['DisType'] == "ADE-Neg":
                bne_count += 1

            if entity['DisType'] == 'Worse':
                worse_count += 1

            if entity['DisType'] == 'NegatedADE':
                negated_ade_count += 1

            if entity['DisType'] == 'Indication':
                adr_count += 1

            if entity['DisType'] == 'Indication' and 'MedDRA' in entity:
                adr_text = entity['MedDRA']
                key = entity['MedDRA']
                if key not in diseases:
                    diseases[key] = []
                diseases[key].append(name)

        names.append(name)
        bne_counts.append(bne_count)
        ade_neg_counts.append(ade_neg_count)
        worse_counts.append(worse_count)
        negated_ade_counts.append(negated_ade_count)
        adr_counts.append(adr_count)
        adr_list.append(adr_text)

In [5]:
#print(sum(adr_counts))
#count = 0
#for elem in adr_list:
#    if elem != 'Нет ADR':
#        count += 1
#print(count)

## Создание таблицы препарат -- рейтинг -- список возможных ADR

### Создание таблицы из списков количества сущностей BNE-Pos, ADE-Neg, Worse, NegatedADE, ADR для каждого препарата

In [6]:
standart_names = standartization(names)

data = {
        'standart_names':standart_names,
        'bne_counts':bne_counts,
        'ade_neg_counts':ade_neg_counts,
        'worse_counts':worse_counts,
        'negated_ade_counts':negated_ade_counts,
        'adr_counts':adr_counts
        }

df_raw = pd.DataFrame(data)
df_drugs = df_raw.groupby('standart_names').sum()

#lst_index = list(df_drugs.index)
#lst_index[0] = 'undefined'
#df_drugs.index = lst_index
df_drugs

,bne_counts,ade_neg_counts,worse_counts,negated_ade_counts,adr_counts
standart_names,,,,,
0,40,0,0,10,22
cold balm,1,0,0,0,7
evergetikon,0,0,0,0,0
агри,0,0,0,5,0
азалептин,3,0,0,0,2
...,...,...,...,...,...
эргоферон,26,0,3,21,34
эреспал,0,0,0,0,0
этиловый спирт,0,0,0,2,0


### Добавление в таблицу количества упоминаний для каждого препарата

In [7]:
df_for_mentions_count = df_raw.groupby('standart_names').count()
number = df_for_mentions_count['bne_counts']
df_drugs.insert(loc=len(df_drugs.columns), column='number_of_mentions', value=number)

### Добавление списка возможных ADR каждого препарата. Список возможных ADR сделан в виде словаря, где для каждого варианта ADR (в том числе для их отсутствия) посчитано количество упоминаний возникновения такого ADR у препарата

In [8]:
data_adr = {
        'standart_names':standart_names,
        'adr_list':adr_list
        }

df_adr = pd.DataFrame(data_adr)
df2_text = df_adr.groupby('standart_names').count()
adr_list_df = list(df2_text['adr_list'])

In [9]:
pointer = 0
adr_list_text = [''] * len(df2_text)
for i in range(len(df2_text)):
    pointer += adr_list_df[i]
    adr_list_text[i] = count_frequency(adr_list[pointer-adr_list_df[i]+1:pointer+1])
df_drugs.insert(loc=len(df_drugs.columns), column='adr_list_text', value=adr_list_text)

### Вычисление рейтинга препаратов. Колонка neg_sum показывает сумму негативных упоминаний для препарата

In [10]:
df_drugs['neg_sum'] = df_drugs['ade_neg_counts'] + df_drugs['worse_counts'] + df_drugs['negated_ade_counts'] + df_drugs['adr_counts']
raiting = [''] * len(df_drugs)
for i in range(len(df_drugs)):
   if df_drugs.iloc[i]['bne_counts'] > df_drugs.iloc[i]['neg_sum']:
       if len(df_drugs.iloc[i]['adr_list_text']) > 1:
           raiting[i] = 'Хороший, но есть ADR'
       else:
           raiting[i] = 'Хороший'
   else:
       if df_drugs.iloc[i]['adr_counts'] > df_drugs.iloc[i]['negated_ade_counts'] and df_drugs.iloc[i]['adr_counts'] > df_drugs.iloc[i]['worse_counts'] and df_drugs.iloc[i]['adr_counts'] > df_drugs.iloc[i]['ade_neg_counts']:
           raiting[i] = 'Плохой: частные побочные реакции'
       elif df_drugs.iloc[i]['negated_ade_counts'] > df_drugs.iloc[i]['adr_counts'] and df_drugs.iloc[i]['negated_ade_counts'] > df_drugs.iloc[i]['worse_counts'] and df_drugs.iloc[i]['negated_ade_counts'] > df_drugs.iloc[i]['ade_neg_counts']:
           raiting[i] = 'Плохой: не помогает'
       else:
           raiting[i] = 'Плохой'
df_drugs.insert(loc=len(df_drugs.columns), column='raiting', value=raiting)

In [11]:
df_drugs_final = df_drugs.drop(['bne_counts', 'ade_neg_counts', 'worse_counts', 'negated_ade_counts', 'adr_counts', 'number_of_mentions', 'neg_sum'], axis=1)
df_drugs_final.columns = ['ADR', 'Raiting']
df_drugs_final

,ADR,Raiting
standart_names,,
0,"{'Нет ADR': 257, 'Стресс': 3, 'Бессонница': 1,...","Хороший, но есть ADR"
cold balm,"{'Нет ADR': 26, 'Нарушение мозгового кровообра...",Плохой: частные побочные реакции
evergetikon,{'Нет ADR': 13},Плохой
агри,{'Нет ADR': 19},Плохой: не помогает
азалептин,"{'Профилактика': 1, 'Нет ADR': 18}","Хороший, но есть ADR"
...,...,...
эргоферон,"{'Нет ADR': 234, 'Боль': 1, 'Пирексия': 1, 'По...",Плохой: частные побочные реакции
эреспал,{'Нет ADR': 1},Плохой
этиловый спирт,{'Нет ADR': 4},Плохой: не помогает


In [12]:
df_drugs_final.to_csv('drugs_table.csv')

In [13]:
df_drugs_final_aggregated = df_drugs_final.groupby('Raiting').count()
df_drugs_final_aggregated.columns = ['Count']
df_drugs_final_aggregated

,Count
Raiting,
Плохой,44
Плохой: не помогает,39
Плохой: частные побочные реакции,82
Хороший,16
"Хороший, но есть ADR",40


In [14]:
df_drugs_final_aggregated.to_csv('drugs_table_aggregated.csv')

## Создание датафрейма формата симптом -- список лучших препаратов

### Создание таблицы вида симптом - список всех нестандартизироавнных названий препаратов.

In [15]:
data_diseases = {
    'disease':diseases.keys(),
    'drug': diseases.values()
}

df_diseases = pd.DataFrame(data_diseases)
df_diseases.set_index('disease', inplace=True)
df_diseases.head(50)

,drug
disease,
Стресс,"[Тенотен, Валерианы экстракт, Валериана, Экстр..."
Бессонница,"[Тенотен, undefined, Сонмил, Афобазол, Афобазо..."
Нервозность,"[Тенотен, Седавит, Вечернее, Тенотен детский, ..."
Эритема глотки,"[Тонзилгон Н, Тонзилгон Н, Эргоферон, Гриппферон]"
Боль в ротоглотке (орофарингеальная),"[Тонзилгон, Тонзилгон, Тонзилгон, Граммидин Не..."
Повышенная температура тела,"[АнтиГриппин, Виферон, Виферон, Виферон, Колдр..."
Нарушение речи,[Кортексин]
Нарушение засыпания,"[Мелаксен, Мелаксен, Мелаксен, Мелаксен]"
Ажитация,"[Экстракт Валерианы, Фенибут, Пустырник, корте..."


In [16]:
for value in diseases.values():
    for elem in value:
        if elem == 0:
            value.remove(elem)

for value in diseases.values():
    print(value)
    for elem in value:
        print(elem)
        if elem == 0:
            print('NULL VALUE!')

['Тенотен', 'Валерианы экстракт', 'Валериана', 'Экстракт Пустырник', 'Седавит', 'Тенотен детский', 'Тенотен детский', 'Пустырник', 'Корвалола', 'Валерианы экстракт', 'валерьянка', 'валерьянку', 'валерьянка', 'валерьянку', 'Барбовал', 'Ацикловир', 'ацикловир', 'Вечернее', 'Вечернее', 'Вечернее', 'Аминофенилмасляная кислота', 'валериановые', 'валериановые', 'Персена', 'Алора', 'Афобазол', 'Афобазол', 'Афобазол', 'Глицин', 'Тенотен детский', 'Глицин', 'Афобазол', 'Корвалдин', 'Тривалумен', 'Тенотен', 'Глицин']
Тенотен
Валерианы экстракт
Валериана
Экстракт Пустырник
Седавит
Тенотен детский
Тенотен детский
Пустырник
Корвалола
Валерианы экстракт
валерьянка
валерьянку
валерьянка
валерьянку
Барбовал
Ацикловир
ацикловир
Вечернее
Вечернее
Вечернее
Аминофенилмасляная кислота
валериановые
валериановые
Персена
Алора
Афобазол
Афобазол
Афобазол
Глицин
Тенотен детский
Глицин
Афобазол
Корвалдин
Тривалумен
Тенотен
Глицин
['Тенотен', 'undefined', 'Сонмил', 'Афобазол', 'Афобазол', 'Афобазол', 'undefined',

In [17]:
diseases.values()

dict_values([['Тенотен', 'Валерианы экстракт', 'Валериана', 'Экстракт Пустырник', 'Седавит', 'Тенотен детский', 'Тенотен детский', 'Пустырник', 'Корвалола', 'Валерианы экстракт', 'валерьянка', 'валерьянку', 'валерьянка', 'валерьянку', 'Барбовал', 'Ацикловир', 'ацикловир', 'Вечернее', 'Вечернее', 'Вечернее', 'Аминофенилмасляная кислота', 'валериановые', 'валериановые', 'Персена', 'Алора', 'Афобазол', 'Афобазол', 'Афобазол', 'Глицин', 'Тенотен детский', 'Глицин', 'Афобазол', 'Корвалдин', 'Тривалумен', 'Тенотен', 'Глицин'], ['Тенотен', 'undefined', 'Сонмил', 'Афобазол', 'Афобазол', 'Афобазол', 'undefined', 'Сонмил', 'Сомнол', 'Сомнол', 'Сомнол', 'undefined', 'Реладорм', 'Глицин', 'Глицин', 'Донормил', 'Вечернее', 'Вечернее', 'Вечернее', 'Настойка пиона уклоняющегося', 'Сондокс', 'Сондокс', 'Сондокс', 'Сондокс', 'undefined', 'Мелаксен', 'undefined', 'undefined', 'Сонмил', 'Азалептин', 'Глицин', 'валерьянку', 'Афобазол', 'Афобазол', 'Тенотен детский', 'undefined', 'Сонмил', 'Сонмил', 'Тенот

### Далее преобразование данных - стандартизация названий препаратов, отбор препаратов только с хорошим рейтингом, сортировка по количеству упоминаний препарата, отбор максимум трех лучших.

In [18]:
def only_good_raiting(list_of_drugs):
    new_list_of_drugs = []
    for drug in list_of_drugs:
        if drug == 0:
            continue
        if df_drugs.loc[drug]['raiting'] in ['Хороший','Хороший, но есть ADR']:
            new_list_of_drugs.append(drug)
    return new_list_of_drugs

def sort_drugs_on_review_number(list_of_drugs):
    dict_of_drugs = {}
    sorted_list = []
    for drug in list_of_drugs:
        dict_of_drugs[drug] = df_drugs.loc[drug]['number_of_mentions']
    sorted_tuples = sorted(dict_of_drugs.items(), key=lambda item: item[1])
    for i in range(len(sorted_tuples)):
        sorted_list.append(sorted_tuples[i][0])
    return sorted_list[::-1]


for key, value in diseases.items():
    diseases[key] = standartization(value)
    diseases[key] = remove_duplicates(diseases[key])
    diseases[key] = only_good_raiting(diseases[key])
    diseases[key] = sort_drugs_on_review_number(diseases[key])
    diseases[key] = top3_drugs(diseases[key])
    if diseases[key] == []:
        diseases[key] = 'Нет хороших лекарств'
    if diseases[key] != 'Нет хороших лекарств':
        diseases[key] = ', '.join(diseases[key])

In [19]:
data_diseases = {
    'disease':diseases.keys(),
    'drug': diseases.values()
}

df_diseases_final = pd.DataFrame(data_diseases)
df_diseases_final.head(50)

,disease,drug
0,Стресс,"персен, тривалумен"
1,Бессонница,"глицисед, азалептин"
2,Нервозность,"глицисед, персен, тривалумен"
3,Эритема глотки,тонзилгон н
4,Боль в ротоглотке (орофарингеальная),"флюколд, тонзилгон н, эвкабал"
5,Повышенная температура тела,"лаферобион, флюколд, орвирем"
6,Нарушение речи,Нет хороших лекарств
7,Нарушение засыпания,Нет хороших лекарств
8,Ажитация,Нет хороших лекарств
9,Психологическое расстройство,глицисед


In [20]:
df_diseases_final.to_csv('diseases_table.csv')